In [1]:
# necessary libraries for pre-processing
import utils
import pandas as pd
import numpy as np
import os
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import preprocessing

## Statistical Analysis

In [2]:
def get_col_max(df, col):
    '''Get the maximum value of a given column'''
    return df[col].max()

def get_col_min(df, col):
    '''Get the minimum value of a given column'''
    return df[col].min()

def get_col_count(df, col):
    '''Get the number of elements of a given column'''
    return df[col].count()

def get_col_avg(df, col):
    '''Get the average value of a given column'''
    return df[col].mean()

def get_col_std(df, col):
    '''Get the standar deviation value of a given column'''
    return df[col].std()

# Useful functions for preprocessing

In [3]:
def convert_date(df, column, date_format='%y%m%d'):
    '''Convert the given column containg dates in the given format
    to the standard date format and type'''
    copy_df = df.copy()
    copy_df[column] = pd.to_datetime(copy_df[column], format=date_format)

    return copy_df

In [4]:
def encode_column(df, column, options_list):
    '''Encode the given column of the given dataframe.
    All column values should be present in the options_list.'''
    copy_df = df.copy()

    le = preprocessing.LabelEncoder()
    le.fit(options_list)
    copy_df[column] = le.transform(copy_df[column])
    
    return copy_df

In [5]:
def normalize_columns(df, columns):
    '''Normalize the given columns for range between [0, 1]'''
    for col in columns:
        col_min = get_col_min(df, col)
        col_max = get_col_max(df, col)
        
        df[col] = (df[col] - col_min)/\
                    (col_max - col_min)

In [6]:
def get_df_correlation(df):
    '''Get the correlation between the dataframe features'''
    # Compute the correlation matrix
    corr = df.corr()

    # Generate a mask for the upper triangle
    mask = np.zeros_like(corr, dtype=np.bool)
    mask[np.triu_indices_from(mask)] = True
    
    # Set up the matplotlib figure
    plt.subplots(figsize=(11, 9))

    # Generate a custom diverging colormap
    cmap = sns.diverging_palette(220, 10, as_cmap=True)

    # Draw the heatmap with the mask and correct aspect ratio
    ax = sns.heatmap(corr, mask=mask, cmap=cmap, center=0,
                     square=True, linewidths=.1, cbar_kws={"shrink": .5})
    
    y_lim = ax.get_ylim();
    ax.set_ylim(np.ceil(y_lim[0]), np.floor(y_lim[1]))

    plt.show()

In [7]:
def scatterplot_two_cols(df, col1, col2):
    '''Get a scatterplot for the given two columns'''
    # Set up the matplotlib figure
    plt.subplots(figsize=(11, 9))
    
    # Generate a custom diverging colormap
    cmap = sns.diverging_palette(220, 10, as_cmap=True)

    sns.scatterplot(data=df, x=col1, y=col2,
                    hue='status',palette=cmap, sizes=(47,47))
    plt.show()

## Imputation

In [8]:
def get_null_summary(dataset):
    '''Get a null summary display'''
    display(dataset.isnull().mean())

In [9]:
def clean_nulls(dataset, threshold=0.7):
    '''Clean nulls from the given table.
    If the nulls in a column are higher than the given threshold the entire column is deleted.
    If the nulls in a row are higher than the row, the row is also deleted.
    The threshold is a value between 0 and 1'''
    #Dropping columns with missing value rate higher than threshold
    dataset = dataset[dataset.columns[dataset.isnull().mean() < threshold]]

    #Dropping rows with missing value rate higher than threshold
    dataset = dataset.loc[dataset.isnull().mean(axis=1) < threshold]
    
    return dataset

In [10]:
def numerical_imputation(dataset, replacer=None):
    '''When null values exist, set them using the median of the colum,
    or a replacer, if one was given'''
    dataset = dataset.fillna(replacer if replacer else dataset.median())

    return dataset

In [11]:
def categorical_imputation(dataset, column_name, replacer=None):
    '''Replace the inexistent values of the given column with the given replacer.
    If None replacer was ginve, use the column maximum value'''
    #Max fill function for categorical columns
    dataset[column_name].fillna(replacer if replacer else \
                                dataset[column_name].value_counts()
                                                    .idxmax(),
                                inplace=True)
    
    return dataset

## Handling Outliers

In [12]:
def display_to_drop_std(dataset, column, mult_factor=3):
    '''Display the rows that will be dropped using the std approach'''
    upper_lim = dataset[column].mean() + dataset[column].std() * mult_factor
    lower_lim = dataset[column].mean() - dataset[column].std() * mult_factor

    display(dataset[(dataset[column] >= upper_lim) & (dataset[column] <= lower_lim)])

def drop_outliers_std(dataset, column, mult_factor=3):
    '''Drop the outlier rows with standard deviation'''
    upper_lim = dataset[column].mean() + dataset[column].std() * mult_factor
    lower_lim = dataset[column].mean() - dataset[column].std() * mult_factor

    return dataset[(dataset[column] < upper_lim) & (dataset[column] > lower_lim)]

In [13]:
def display_to_drop_percentile(dataset, column):
    '''Display the rows that will be dropped with Percentiles approach'''
    upper_lim = dataset[column].quantile(.95)
    lower_lim = dataset[column].quantile(.05)

    display(dataset[(dataset[column] >= upper_lim) & (dataset[column] <= lower_lim)])

def drop_outliers_percentile(dataset, column):
    '''Drop the outlier rows with Percentiles approach'''
    upper_lim = dataset[column].quantile(.95)
    lower_lim = dataset[column].quantile(.05)

    data = dataset[(dataset[column] < upper_lim) & (dataset[column] > lower_lim)]

## Binning

In [14]:
def numerical_binning(dataset):
    # TODO
    # https://towardsdatascience.com/feature-engineering-for-machine-learning-3a5e293a5114
    return False

In [15]:
def categorical_binning(dataset):
    # TODO
    return False

# Feature Engineering

### For a first simpler approach, we will only use the 'loan' table

In [16]:
# Loading the different train tables
loan_df = utils.read_csv_to_df('competition_dataset/loan_train.csv', delimiter=';')
account_df = utils.read_csv_to_df('competition_dataset/account.csv', delimiter=';')
disp_df = utils.read_csv_to_df('competition_dataset/disp.csv', delimiter=';')
card_df = utils.read_csv_to_df('competition_dataset/card_train.csv', delimiter=';')
trans_df = utils.read_csv_to_df('competition_dataset/trans_train.csv', delimiter=';')

# Loading the test tables
loan_test_df = utils.read_csv_to_df('competition_dataset/loan_test.csv', delimiter=';')
card_test_df = utils.read_csv_to_df('competition_dataset/card_test.csv', delimiter=';')
trans_test_df = utils.read_csv_to_df('competition_dataset/trans_test.csv', delimiter=';')


print(' ::: Tables Scheme :::')
print('\n\n\t LOAN TABLE')
display(loan_df.head())
print('\n\n\t ACCOUNT TABLE')
display(account_df.head())
print('\n\n\t DISPOSITION TABLE')
display(disp_df.head())
print('\n\n\t CREDIT CARD TABLE')
display(card_df.head())
print('\n\n\tTRANSACTIONS TABLE')
display(trans_df.head())

 ::: Tables Scheme :::


	 LOAN TABLE


/Users/edgarcarneiro/Documents/University/feup-ecac/project-competition/env/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3249: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


,loan_id,account_id,date,amount,duration,payments,status
0,5314,1787,930705,96396,12,8033,-1
1,5316,1801,930711,165960,36,4610,1
2,6863,9188,930728,127080,60,2118,1
3,5325,1843,930803,105804,36,2939,1
4,7240,11013,930906,274740,60,4579,1




	 ACCOUNT TABLE


,account_id,district_id,frequency,date
0,576,55,monthly issuance,930101
1,3818,74,monthly issuance,930101
2,704,55,monthly issuance,930101
3,2378,16,monthly issuance,930101
4,2632,24,monthly issuance,930102


...

,disp_id,client_id,account_id,type
0,1,1,1,OWNER
1,2,2,2,OWNER
2,3,3,2,DISPONENT
3,4,4,3,OWNER
4,5,5,3,DISPONENT




	 CREDIT CARD TABLE


,card_id,disp_id,type,issued
0,1005,9285,classic,931107
1,104,588,classic,940119
2,747,4915,classic,940205
3,70,439,classic,940208
4,577,3687,classic,940215




	TRANSACTIONS TABLE


,trans_id,account_id,date,type,operation,amount,balance,k_symbol,bank,account
0,1548749,5270,930113,credit,credit in cash,800.0,800.0,NaN,NaN,NaN
1,1548750,5270,930114,credit,collection from another bank,44749.0,45549.0,NaN,IJ,80269753.0
2,3393738,11265,930114,credit,credit in cash,1000.0,1000.0,NaN,NaN,NaN
3,3122924,10364,930117,credit,credit in cash,1100.0,1100.0,NaN,NaN,NaN
4,1121963,3834,930119,credit,credit in cash,700.0,700.0,NaN,NaN,NaN


# Individual pre processement of tables

In [17]:
def process_loans(loans_df, debug=False):
    '''Pre process the loan table'''
    if debug:
        print(' > Raw loan table representation')
        display(loans_df)
        print(' > Raw loan table correlations')
        get_df_correlation(loans_df)
        
    processed_df = convert_date(loan_df, 'date')
    
    # As there is a correlation between amount, duration & payments
    if debug:
        print(' > There is naturally a correlation between the amount, duration and payments columns,'+
              'since amount = columns * payments')
        scatterplot_two_cols(processed_df, 'amount', 'duration')
        scatterplot_two_cols(processed_df, 'amount', 'payments')
        print(' > Natural correlations, since the larger the amount the larger the monthly payment or the duration.')
        
    return df

In [18]:
def process_account(accounts_df, debug=False):
    '''Pre process the accounts table'''
    if debug:
        print(' > Raw accounts table representation')
        display(accounts_df)
        print(' > Raw accounts table correlations')
        get_df_correlation(accounts_df)
        
    processed_account = convert_date(account_df, 'date')
    
    # Renaming account attributes
    processed_account.loc[processed_account["frequency"]=="monthly issuance","frequency"] = "MI"
    processed_account.loc[processed_account["frequency"]=="weekly issuance","frequency"] = "WI"
    processed_account.loc[processed_account["frequency"]=="issuance after transaction","frequency"] = "IAT"
    
    # Transform numerical into categorical
    df = pd.get_dummies(processed_account)
    
    if debug:
        print(' > Transformed the categorical frequency column into numerical respective columns')
        display(df)
        print(' > Since the 3 categories are depedent, we can remove one of them, for removing redundant data')
        
    df = df.drop(['frequency_IAT'], axis=1)
        
    if debug:
        print(' > Hence, we get the processed accounts dataframe:')
        display(df)
        print(' > And the features correlation:')
        get_df_correlation(df)

    return df

In [19]:
def process_dispostition(disp_df, debug=False):
    '''Pre process the dispositions table'''
    if debug:
        print(' > Raw disposition table representation')
        display(disp_df)
        print(' > Raw disposition table correlations')
        get_df_correlation(disp_df)
        
    processed_disp = disp_df.copy()
    
    # Renaming disp attributes
    processed_disp.loc[processed_disp["type"]=="OWNER","type"] = "O"
    processed_disp.loc[processed_disp["type"]=="DISPONENT","type"] = "U"
    
    # Transform numerical into categorical
    df = pd.get_dummies(processed_disp)
    
    if debug:
        print(' > Transformed the categorical type column into numerical respective columns')
        display(df)
        print(' > Since the 2 categories are depedent, we can remove one of them, for removing redundant data')
        get_df_correlation(df)
        
    # Cannot process further as needs merging with other columns
    return df

In [20]:
def process_card(card_df, debug=False):
    '''Pre process the credit card table'''
    if debug:
        print(' > Raw credit card table representation')
        display(card_df)
        print(' > Raw credit card table correlations')
        get_df_correlation(card_df)

    processed_card = convert_date(card_df, 'issued')

    # Renaming card attributes
    processed_card.loc[processed_card["type"]=="classic","type"] = "C"
    processed_card.loc[processed_card["type"]=="gold","type"] = "G"
    processed_card.loc[processed_card["type"]=="junior","type"] = "J"
    
    # Transform numerical into categorical & removing useless column
    df = pd.get_dummies(processed_card)
    df = df.drop(['card_id'], axis=1)
    
    
    if debug:
        print(' > Transformed the categorical type column into numerical respective columns')
        display(df)
        print(' > Since the 2 categories are depedent, we can remove one of them, for removing redundant data')
        get_df_correlation(df)
        
    # Cannot process further as needs merging with other columns
    return df

In [21]:
def process_transactions(transactions_df, debug=False):
    '''Pre process the transactions table'''
    if debug:
        print(' > Raw transactions table representation')
        display(transactions_df)
        print(' > Raw transactions table correlations')
        get_df_correlation(transactions_df)
        
    if debug:
        print(' > Null evaluation in transactions rows')
        get_null_summary(transactions_df)
        
    # Removing the null columns & processing data
    processed_df = clean_nulls(transactions_df)
    processed_df = convert_date(processed_df, 'date')
    
    # Renaming 'withdrawal in cash' to 'withdrawal'
    processed_df.loc[processed_df["type"]=="withdrawal in cash","type"] = "withdrawal"
    
    if debug:
        display(processed_df['operation'].unique())
        display(processed_df['type'].unique())
        display(processed_df['k_symbol'].unique())
        display(processed_df.sort_values(by=['account_id', 'date'],
                                            ascending=[True, False]))

    # Aggregatting transaction balances
    agg_ballance = processed_df.sort_values(by=['account_id', 'date'],
                                            ascending=[True, False])\
                               .groupby(['account_id'])\
                               .agg({
                                    'balance': ['mean', 'max', 'min', lambda x: x.iloc[0]]
                               })\
                               .reset_index()
    agg_ballance.columns = ['account_id', 'balance_mean', 'balance_max', 'balance_min', 'last_ballance']

    # Agrregatting credits
    agg_credits = processed_df.groupby(['account_id', 'type'])\
                              .agg({
                                  'amount': ['mean', 'count', 'max', 'min'],
                              })\
                              .reset_index()
    agg_credits.columns = ['account_id', 'type', 'credit_mean', 'credit_count', 'credit_max', 'credit_min']
    agg_credits = agg_credits[agg_credits['type'] == 'credit']
    
    # Aggregatting withdrawals
    agg_withdrawals = processed_df.groupby(['account_id', 'type'])\
                                  .agg({
                                    'amount': ['mean', 'count', 'max', 'min'],
                                  })\
                                  .reset_index()
    agg_withdrawals.columns = ['account_id', 'type', 'withdrawal_mean', 'withdrawal_count',
                           'withdrawal_max', 'withdrawal_min']
    agg_withdrawals = agg_withdrawals[agg_withdrawals['type'] == 'withdrawal']

    # Aggregatting all the 3 tables into one
    df = agg_ballance.merge(agg_credits, on='account_id')\
                          .merge(agg_withdrawals, on='account_id', how='left')
    
    # Cleaning nulls and useless columns
    df = df.drop(['type_x', 'type_y'], axis=1)
    df = numerical_imputation(df, 0)
    
    # Adding extra columns
    df['mean_trans_profit'] = df['credit_mean'] - df['withdrawal_mean']

    if debug:
        print(' > Table after processment of balance, credits and withdrawals')
        display(df)
        get_null_summary(df)
        print(' > And the corresponding correlation matrix')
        get_df_correlation(df)
    
    return df

# Composite pre processment of tables

In [ ]:
def compose_dataset(loan_df, account_df, disp_df, card_df, debug=False):
    '''Join the different tables and apply feature engineering'''

    # Creating formatted copy tables, so I can edit without changing original
    processed_loan = convert_date(loan_df, 'date')
    processed_disp =
    processed_card = 




    # Joining loan table, accounts table, disposition table & card table
    df = processed_loan.merge(processed_account.rename(columns={'date': 'account_creation_date'}),
                              on='account_id')\
                       .merge(processed_disp, on='account_id')\
                       .merge(processed_card.rename(columns={'type': 'card_type'}),
                              on='disp_id', how='left')
    
    if debug:
        print(' > Joint tables analysis')
        get_df_correlation(df)
    
    # Getting loanes age, in months & duration in months as well
    df['account_loan_age'] = (df['date'] - df['account_creation_date']).dt.days / 31
    df["date"] = pd.to_numeric(df["date"])
    df['duration'] = df['duration'] / 31

    # Separating categorial features
    df = pd.get_dummies(df)
    
    # Getting the Number of account owners & type of cards related to each account
    df_num_owners_and_types = df.groupby(
        ['loan_id', 'account_id']
    ).agg({
        'type_O': ['count'],
        'card_type_C': ['sum'],
        'card_type_G': ['sum'],
        'card_type_J': ['sum']
    }).reset_index()
    df_num_owners_and_types.columns = ['loan_id', 'acc', 'owners_count', 'C', 'G', 'J']
    df_num_owners_and_types = df_num_owners_and_types.drop(['acc'], axis=1)

    # Dropping useless columns
    df = df.drop(['account_creation_date', 'account_id', 'district_id',
                  'frequency_IAT', 'type_U', 'type_O', 'disp_id',
                  'client_id', 'card_id', 'issued', 'card_type_C',
                  'card_type_G', 'card_type_J'], axis=1)\
           .drop_duplicates()
    
    # Merging with the aggregattion results
    df = df.merge(df_num_owners_and_types, on='loan_id')
    df = df.drop(['J'], axis=1)
    
    # Normalize numerical columns
    normalize_columns(df, ['amount', 'payments', 'date', 'monthly_loan',
                           'monthly_loan-to-monthly_receiving', 'monthly_only_receiving'])

    # Writing loan_id for later association on predictions
    utils.write_df_to_csv(
        df[['loan_id', 'date', 'amount']],
        'dataset', 'ids.csv')
    df = df.drop(['loan_id', 'duration', 'num_payments', 'months_of_receiving', 'G'], axis=1)

    # Placing status column as last column
    return  df[[col for col in df if col not in ['status']] + ['status']]


dataset = compose_dataset(loan_df, account_df, disp_df, card_df, debug=True)
display(dataset)
get_df_correlation(dataset)

In [ ]:
# Good tutorial for feature engineering:
# https://medium.com/datadriveninvestor/a-simple-guide-to-creating-predictive-models-in-python-part-1-8e3ddc3d7008

In [ ]:
# Outputting the resultant table to a final csv
utils.write_df_to_csv(dataset, 'dataset', 'preprocessed_data.csv')
utils.write_df_to_csv(compose_dataset(loan_test_df, account_df, disp_df, card_test_df),
                      'dataset', 'test_dataset.csv')